<a href="https://colab.research.google.com/github/RohanCoderiiitb/TinyML-Harvard/blob/main/Malaria_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
tf_version = "2.14.0"
hub_version = "0.15.0"
datasets_version = "4.6.0"
numpy_version = "1.26.4"
protobuf_version = "3.20.3"

import tensorflow as tf
import numpy as np

try:
    import tensorflow_hub as hub
    import tensorflow_datasets as tfds
except:
    hub = None
    tfds = None

# Check if versions match
if (tf.__version__ != tf_version or
    (hub and hub.__version__ != hub_version) or
    (tfds and tfds.__version__ != datasets_version) or
    not np.__version__.startswith(numpy_version)):

    print(f"Current TensorFlow version: {tf.__version__} → {tf_version}")
    if hub: print(f"Current TensorFlow Hub version: {hub.__version__} → {hub_version}")
    if tfds: print(f"Current TensorFlow Datasets version: {tfds.__version__} → {datasets_version}")
    print(f"Current NumPy version: {np.__version__} → {numpy_version}")

    # Uninstall old versions
    !pip uninstall -y tensorflow tensorflow_hub tensorflow_datasets numpy protobuf

    # Install specific versions
    !pip install tensorflow=={tf_version} \
                tensorflow_hub=={hub_version} \
                tensorflow_datasets=={datasets_version} \
                numpy=={numpy_version} \
                protobuf=={protobuf_version}

    # Prompt user to restart
    print("\n Specified versions installed successfully.")
    print(" Please restart the runtime (Runtime > Restart session) and re-run the notebook.\n")
else:
    print(" All packages are already at the specified versions.")

Current TensorFlow version: 2.18.0 → 2.14.0
Current TensorFlow Hub version: 0.16.1 → 0.15.0
Current TensorFlow Datasets version: 4.9.9 → 4.6.0
Current NumPy version: 2.0.2 → 1.26.4
Found existing installation: tensorflow 2.18.0
Uninstalling tensorflow-2.18.0:
  Successfully uninstalled tensorflow-2.18.0
Found existing installation: tensorflow-hub 0.16.1
Uninstalling tensorflow-hub-0.16.1:
  Successfully uninstalled tensorflow-hub-0.16.1
Found existing installation: tensorflow-datasets 4.9.9
Uninstalling tensorflow-datasets-4.9.9:
  Successfully uninstalled tensorflow-datasets-4.9.9
Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
Found existing installation: protobuf 5.29.5
Uninstalling protobuf-5.29.5:
  Successfully uninstalled protobuf-5.29.5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of tensorflow-metadata to determine which version is compatible w


 Specified versions installed successfully.
 Please restart the runtime (Runtime > Restart session) and re-run the notebook.



In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub
import numpy as np

# Fetching and Preparing Data

In [2]:
def format_image(image, label):
  image = tf.image.resize(image, (224, 224))/255.0
  return image, label

(raw_train, raw_val, raw_test), metadata = tfds.load(
    'malaria',
    split=["train[:80%]", "train[80%:90%]", "train[90%:]"],
    with_info=True,
    as_supervised=True
)
NO_OF_EXAMPLES = metadata.splits['train'].num_examples
NO_OF_CLASSES = metadata.features['label'].num_classes
NO_OF_BATCHES = 64
NO_OF_SHUFFLES = NO_OF_EXAMPLES//4
AUTOTUNE = tf.data.AUTOTUNE
train_batches = raw_train.shuffle(NO_OF_SHUFFLES).map(format_image).batch(NO_OF_BATCHES).prefetch(AUTOTUNE)
val_batches = raw_val.map(format_image).batch(NO_OF_BATCHES).prefetch(AUTOTUNE)
test_batches = raw_test.map(format_image).batch(1)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/27558 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/malaria/1.0.0.incompleteQ58LGJ/malaria-train.tfrecord*...:   0%|          | 0/…

Dataset malaria downloaded and prepared to ~/tensorflow_datasets/malaria/1.0.0. Subsequent calls will reuse this data.


In [3]:
for image_batch, label_batch in train_batches.take(1):
  pass
image_batch.shape

TensorShape([64, 224, 224, 3])

# Training the Model

1. Using a pre-trained model - MobileNet_V2

In [4]:
model_selection = ("mobilenet_v2", 224, 1280)
handle_base, pixels, fvsize = model_selection
model_handle = "https://tfhub.dev/google/tf2-preview/{}/feature_vector/4".format(handle_base)

model = tf.keras.Sequential([
  hub.KerasLayer(
      model_handle,
      input_shape=(pixels, pixels, 3),
      output_shape=[fvsize],
      trainable=False
  ),
  tf.keras.layers.Dense(NO_OF_CLASSES, activation='softmax')
])

model.summary()

model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=['accuracy'])

model.fit(
    train_batches,
    epochs=3,
    validation_data=val_batches
)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              2257984   
                                                                 
 dense (Dense)               (None, 2)                 2562      
                                                                 
Total params: 2260546 (8.62 MB)
Trainable params: 2562 (10.01 KB)
Non-trainable params: 2257984 (8.61 MB)
_________________________________________________________________
Epoch 1/3
345/345 [==============================] - 1257s 4s/step - loss: 0.2258 - accuracy: 0.9119 - val_loss: 0.1690 - val_accuracy: 0.9412
Epoch 2/3
345/345 [==============================] - 1023s 3s/step - loss: 0.1699 - accuracy: 0.9404 - val_loss: 0.1558 - val_accuracy: 0.9412
Epoch 3/3
345/345 [==============================] - 946s 3s/step - loss: 0.1570 - accuracy: 0.9445 - val_loss: 0.1544 - val_a

In [5]:
export_dir = "SavedModels/Model1"
tf.saved_model.save(model, export_dir)

In [6]:
import pathlib
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
tflite_model = converter.convert()

tflite_model_dir = pathlib.Path("/tmp")
tflite_model_file = tflite_model_dir/"model1.tflite"
tflite_model_file.write_bytes(tflite_model)

8875280

In [7]:
from tqdm import tqdm
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

correct = 0
total = 0

for image, label in tqdm(test_batches):
  interpreter.set_tensor(input_index, image)
  interpreter.invoke()
  output = interpreter.get_tensor(output_index)
  prediction = np.argmax(output)
  actual = label.numpy()[0]
  if(prediction==actual):
    correct+=1
  total+=1
accuracy = correct/total
print("Accuracy: {:.2f}%".format(accuracy*100))

100%|██████████| 2756/2756 [01:21<00:00, 33.63it/s]

Accuracy: 94.30%
